# RAG LANGCHAIN + Gemini + Nomic + CHROMADB

Para este proyecto, se necesita las siguientes dependencias: 
* Streamlit
* Langchain_community
* Lanchain_core
* langchain_ollama (Embeddings)
* langchain-google-genai (Gemini)
* python-dotenv


pip install protobuf==4.25.3 grpcio==1.60.0 langchain langchain-community langchain-core langchain-google-genai langchain-ollama google-generativeai streamlit chromadb 


In [1]:
import streamlit as st
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
import os
import hashlib

### Custom prompts 
Prompt que sera enviada en cada peticion hacia el modelo, por lo tanto, no debe ser tan larga, debe ser concisa y con el suficiente contexto para trabajar.

In [ ]:
custom_template = """
Actúa como asistente educativo que genera preguntas de opción múltiple adaptativas.

INPUTS:
- Materia
- Unidad Temática
- Evidencia: Conocimiento | Procedimiento | Producto
- Nivel: 1=Basico-Bajo | 2=Basico | 3=Satisfactorio | 4=Avanzado

OUTPUT:
- SOLO JSON válido con esta estructura:

{{
  "Titulo": "string ≤80",
  "Consigna": "pregunta clara",
  "Contexto": "string ≤200",
  "Dificultad": "Basico-Bajo" | "Basico" | "Medio" | "Alto",
  "TiempoEstimado": "MM:SS (01:00-02:00)",
  "VectorNivelOpciones": {{
    "OpcionA": ["Bajo", "Medio", "Alto", "Alto"],
    "OpcionB": ["Medio", "Medio", "Bajo", "Alto"],
    "OpcionC": ["Alto", "Bajo", "Medio", "Medio"],
    "OpcionD": ["Medio", "Alto", "Bajo", "Bajo"]
  }},
  "Opciones": {{ "A": "str", "B": "str", "C": "str", "D": "str" }},
  "RespuestaCorrecta": "A"|"B"|"C"|"D"
}}

REGLAS:
- Una única respuesta correcta, 3 distractores plausibles.
- Lenguaje acorde al nivel.
- Consigna = pregunta directa alineada con la evidencia.
- Contexto breve y realista.
- Opciones similares en longitud y sin pistas.
"""

### Directorio de PDFs ya utilizados para el embeddings

In [3]:
pdf_directory = "./data"
db_directory = "./db"

if not os.path.exists(db_directory):
    os.makedirs(db_directory)
if not os.path.exists(pdf_directory):
    os.makedirs(pdf_directory)



### Configuración del Modelo de Embeddings.
Esta es la configuración para aplicar el modelo de embeddings con ollama.
El modelo a utilizar es ***nomic-embed-text.***

In [4]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory=db_directory,
)


C:\Users\gabri\AppData\Local\Temp\ipykernel_37636\3662783597.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


### Procesamiento de los PDFs para embeddings

In [5]:
def upload_pdf(file):
    with open(pdf_directory + file.name, "wb") as f:
        f.write(file.getbuffer())
        
def load_pdf(file):
    loader = PDFPlumberLoader(file)
    documents = loader.load()
    return documents

def text_splitter(documents, course_name):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
    )    
    chunks = text_splitter.split_documents(documents)
    if course_name:
        for i, doc in enumerate(chunks):
            doc.metadata["course_name"] = course_name
            chunks.page_content = f"Curso: {course_name}\n" + doc.page_content
    return chunks


def index_docs(documents):
    vectorstore.add_documents(documents)
    vectorstore.persist()
    print("Documents indexed successfully. Numbers of documents:", len(documents))

### Retrieve docs
Recuperación los documentos por busqueda de similitudes.

In [6]:
def retrieve_docs(query, course_name):
    docs = vectorstore.similarity_search(query, k=5)
    print("Retrieved documents:", len(docs))
    if course_name:
        docs = [doc for doc in docs if doc.metadata.get("course_name") == course_name]
    else:
        docs = [doc for doc in docs]
    if not docs:
        print("No documents found for the given course name.")
    return docs

### Funcion para obtener el hash del documento

Esta funcion sirve para corroborar de que no fue vectorizado aun

In [7]:
def get_file_hash(file_path):
    hasher = hashlib.sha256()
    with open(file_path, "rb") as f:
        buf = f.read()
        hasher.update(buf)
    return hasher.hexdigest()

def is_pdf_already_indexed(file_path):
    result = vectorstore.similarity_search(file_path, k=1)
    if result:
        for doc in result:
            if doc.metadata.get("file_hash") == get_file_hash(file_path):
                return True
    return False

### Función para autentificar el modelo. 
En este caso, se utiliza genai, la libreria de google para comunicarse con Gemini Ai.

In [8]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("API_KEY")

if not api_key:
    st.error("API key not found. Please set the API_KEY environment variable.")
    


In [9]:
import os

os.environ["GOOGLE_API_KEY"] = api_key

### Funcion para obtener la respuesta de Gemini en formato Stream

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

def generate_response_stream(context, course_title, lesson_title, instructor_request, amount):
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.2,
        max_tokens=2000,
        top_p=0.9,
        )
    prompt = ChatPromptTemplate.from_messages([
        ("system", custom_template),
        ("user", "{course_title} {lesson_title} {instructor_request} {contexto} {amount}"),
        ],
    )

    chain = prompt | llm | StrOutputParser()
    
    input_dict = {
        "course_title": course_title,
        "lesson_title": lesson_title,
        "instructor_request": instructor_request,
        "contexto": context,
        "amount": amount
    }
    print("Input dictionary:", input_dict)
    
    for chunk in chain.stream(input_dict):
        yield chunk 

uploaded_file = st.file_uploader("Sube un archivo PDF", type="pdf")
name_course = st.text_input("Nombre del curso")

  



c:\Users\gabri\Desktop\rag\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-18 16:07:12.251 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-18 16:07:12.253 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-18 16:07:12.254 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-18 16:07:12.258 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-18 16:07:12.821 
  command:

    stream

### Visualizar la interfaz con STREAMLIT para demo

### Inputs para el embeddings
* Nombre del curso
* El documento.

In [30]:
uploaded_file = st.file_uploader("Sube un archivo PDF", type="pdf")
name_course = st.text_input("Nombre del curso")

2025-05-13 16:19:50.657 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-13 16:19:50.660 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-13 16:19:50.661 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-13 16:19:50.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-13 16:19:50.769 
  command:

    streamlit run c:\Users\gabri\Desktop\RAG - Gemini\env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-13 16:19:50.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-13 16:19:50.772 Thre

### Realizar el embeddings
Cuando un pdf es subido, se realiza el procesamiento y vectorizacion

In [ ]:
if uploaded_file and name_course:
    upload_pdf(uploaded_file)
    documents = load_pdf(pdf_directory + uploaded_file.name)

    file_hash = get_file_hash(pdf_directory + uploaded_file.name)
    if is_pdf_already_indexed(file_hash):
        st.warning("Este PDF ya ha sido indexado.")
    else:
        chunked_documents = text_splitter(documents)
        for doc in chunked_documents:
            doc.metadata["file_hash"] = file_hash
            doc.metadata["course_name"] = name_course
            print("--- Documento a indexar ---")
            print(doc.page_content)
            print(doc.metadata)
        index_docs(chunked_documents)
        st.success("PDF subido y procesado correctamente.")

### Inputs para el instructor

In [ ]:

course_title = st.text_input("Titulo del curso")
lesson_title = st.text_input("Titulo de la lección")
instructor_request = st.text_area("Ordenes para el asistente")
amount = st.number_input("Cantidad de ejercicios a generar")
    

### Petición al modelo

In [ ]:
if course_title != "" and lesson_title != "" and instructor_request != "" and amount != "":
    st.chat_message("user").write(f"Título del curso: {course_title}")
    st.chat_message("user").write(f"Título de la lección: {lesson_title}")
    st.chat_message("user").write(f"Pedido del instructor: {instructor_request}")
    st.chat_message("user").write(f"Cantidad de ejercicios: {amount}")
    st.markdown("---")

  
    related_documents = retrieve_docs(lesson_title, course_title)


    contexto = "\n".join(doc.page_content for doc in related_documents) if related_documents else ""

    message_placeholder = st.chat_message("assistant").empty()
    full_response = ""

    for chunk in generate_response_stream(contexto, course_title, lesson_title ,instructor_request, amount):
        full_response += chunk  # cada chunk trae parte del texto
        message_placeholder.markdown(full_response)  
